In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
import tqdm
import plotly
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter

## Experiment 2

In [50]:
# download data from wandb (https://wandb.ai/nauqs/experiments-energy)
api = wandb.Api()

# get the runs
runs = list(api.runs("nauqs/experiment-2")) + list(api.runs("nauqs/experiment-2-b"))

len(runs)

20

In [51]:
metrics = ["average_length", "average_return", "average_eat_count", "average_mix_rate", "average_agent_distance"]

# list to store DataFrames for each run
df_list = []
for run in tqdm.tqdm(runs):
    scan_hist = run.scan_history(keys=["timestep", *metrics], page_size=10000000, min_step=0)
    run_df = pd.DataFrame([val for val in scan_hist])
    run_df["run_id"] = run.id
    df_list.append(run_df)

# concatenate all DataFrames
metrics_df = pd.concat(df_list, ignore_index=True)
config = pd.DataFrame([r.config for r in runs])
config["run_id"] = [r.id for r in runs]
# do a joint to have time_bonus and box_reward for each run in metrics_df using run_id
metrics_df = metrics_df.merge(config[["run_id", "time_bonus", "box_reward", "env_id", "gae_lambda"]], on="run_id")
metrics_df["agent"] = metrics_df["time_bonus"].apply(lambda x: "A" if x == 0 else "B")

metrics_df_b = metrics_df[metrics_df.gae_lambda == 1.]
metrics_df = metrics_df[metrics_df.gae_lambda == 0.95]

100%|██████████| 20/20 [00:19<00:00,  1.05it/s]


In [43]:
metrics_df_b.head()

,timestep,average_length,average_return,average_eat_count,average_mix_rate,average_agent_distance,run_id,time_bonus,box_reward,env_id,gae_lambda,agent
219600,0.0,108.906250,0.890630,0.890625,-0.078125,12.468750,798y5z2b,0.1,0,EnergyBoxesDelay,1.0,B
219601,16384.0,110.152672,1.015273,1.160305,-0.117684,14.419847,798y5z2b,0.1,0,EnergyBoxesDelay,1.0,B
219602,32768.0,110.615385,1.061544,1.253846,-0.211026,14.261538,798y5z2b,0.1,0,EnergyBoxesDelay,1.0,B
219603,49152.0,109.236641,0.923670,1.022901,-0.166667,14.030534,798y5z2b,0.1,0,EnergyBoxesDelay,1.0,B
219604,65536.0,109.692308,0.969236,1.100000,-0.229560,14.715385,798y5z2b,0.1,0,EnergyBoxesDelay,1.0,B


In [52]:
metrics_to_plot = ["average_length", "average_eat_count", "average_mix_rate", "average_agent_distance"]
agent_legend = {"A": "A (goal reward)", "B": "B (time bonus)"}

# colors grey (agent A) and golden (agent B)
colors = {"A": "rgba(128, 128, 128, 0.2)", "B": "rgba(221, 167, 0, 0.2)"}
line_colors = {"A": "rgb(128, 128, 128)", "B": "rgb(221, 167, 0)"}
yaxis_max = {"average_length": 256,
                "average_return": 30,
                "average_eat_count": 50,
                "average_mix_rate": 1.02,
                "average_agent_distance": 50}

metrics_legend = {"average_length": "Episode length",
                "average_return": "Episode return",
                "average_eat_count": "Number of boxes eaten",
                "average_mix_rate": "Mix rate",
                "average_agent_distance": "Agent distance"}

window_length = 11  # Length of the filter window (must be a positive odd integer).
polyorder = 2  # Order of the polynomial used to fit the samples.

In [53]:
pd.options.mode.chained_assignment = None

In [54]:
def plot_metrics_multiplot(df, envs, image_name, plot_runs=False, opacity=0.2, width=0.5, window_length=11):

    # use plotly to plot the data
    fig = make_subplots(
        rows=len(envs),
        cols=len(metrics_to_plot),
        subplot_titles=[metrics_legend[metric] for _, _ in envs for metric in metrics_to_plot],
        horizontal_spacing=0.05,
        vertical_spacing=0.1, 
    )

    for j, (env_id, yaxis_max) in enumerate(envs):
        runs_df = df[df["env_id"]==env_id]

        max_values = runs_df.groupby('timestep')[metrics].max()
        
        # compute median and std for each metric grouped by agent column
        env_df = runs_df[["agent", "timestep"] + metrics].groupby(["timestep", "agent"]).agg(["median", "mean", "std"]).reset_index()

        # Additional submetrics
        submetrics = ["median", "std", "mean", "upper", "lower"]

        # Apply smoothing to all metric columns for each agent
        for agent in ["A", "B"]:
            agent_df = env_df[env_df['agent'] == agent]
            for metric in metrics:
                agent_df[(metric, "upper")] = np.minimum(agent_df[(metric, "median")] + agent_df[(metric, "std")], max_values[metric].values)
                agent_df[(metric, "lower")] = np.maximum(agent_df[(metric, "median")] - agent_df[(metric, "std")], 0) 

                for submetric in submetrics:
                    y = agent_df[metric][submetric].values
                    y_smooth = savgol_filter(y, window_length*(2 if submetric in ["upper", "lower"] else 1), polyorder)

                    env_df.loc[agent_df.index, (metric, submetric)] = y_smooth

        for i, metric in enumerate(metrics_to_plot):
            for agent in ["A","B"]:
                agent_df = env_df[env_df['agent'] == agent]
                agent_runs_df = runs_df[runs_df["agent"] == agent]

                if plot_runs:
                    for run_id in sorted(agent_runs_df['run_id'].unique()):
                        run_df = agent_runs_df[agent_runs_df['run_id'] == run_id]
                        y_smoothed = savgol_filter(run_df[metric].values, window_length, polyorder)

                        fig.add_trace(
                            plotly.graph_objects.Scatter(
                                x=run_df["timestep"],
                                y=y_smoothed,
                                line=dict(color=line_colors[agent], width=width),
                                name=f"Agent {agent}",
                                mode='lines',
                                line_shape='spline',   
                                showlegend=False,
                                opacity=opacity,
                            ),
                            row=j+1,
                            col=i+1,
                        )

                showlegend = True if i == 0 and j == 0 else False 

                # median
                fig.add_trace(
                    plotly.graph_objects.Scatter(
                        x=agent_df["timestep"],
                        y=agent_df[metric]["median"],
                        line=dict(color=line_colors[agent]),
                        name=f"{agent_legend[agent]}",
                        mode='lines',
                        line_shape='spline',   
                        showlegend=showlegend 
                    ),
                    row=j+1,
                    col=i+1,
                )

                if plot_runs:
                    # mean
                    fig.add_trace(
                        plotly.graph_objects.Scatter(
                            x=agent_df["timestep"],
                            y=agent_df[metric]["mean"],
                            line=dict(color=line_colors[agent], dash='dot'),
                            name=f"{agent_legend[agent]}",
                            mode='lines',
                            line_shape='spline',   
                            showlegend=showlegend  # Add this line
                        ),
                        row=j+1,
                        col=i+1,
                    )

                # std area
                if not plot_runs:
                    fig.add_trace(
                        plotly.graph_objects.Scatter(
                            x=list(agent_df["timestep"]) + list(agent_df["timestep"])[::-1],
                            y=list(agent_df[metric]["upper"]) + list(agent_df[metric]["lower"])[::-1],
                            fill='toself',
                            fillcolor=colors[agent],
                            line_color='rgba(255,255,255,0)',
                            showlegend=False,
                            mode='lines',
                            line_shape='spline',   
                        ),
                        row=j+1,
                        col=i+1,
                    )
                
            fig.update_yaxes(range=[-0.02*(yaxis_max[metric]), yaxis_max[metric]], row=j+1, col=i+1)
            fig.update_xaxes(range=[0, 20000000], row=j+1, col=i+1, title="Timestep" if j == len(envs) - 1 else None)

    annotations = []
    # Add metric subtitles to the first row
    for i, metric in enumerate(metrics_to_plot):
        annotations.append(dict(xref='paper', yref='paper', x=1.05*(i+0.5)/len(metrics_to_plot)-0.03, y=1.01,
                                xanchor='center', yanchor='bottom',
                                text=metrics_legend[metric], showarrow=False,
                                font=dict(size=18)))
    
    # Add vertical titles
    for j, (env_id, _) in enumerate(envs):
        annotations.append(dict(xref='paper', yref='paper', x=-0.04, y=1.1*((len(envs)-j-0.5)/len(envs))-0.05,
                                xanchor='center', yanchor='middle',
                                text=env_id, showarrow=False,
                                textangle=-90, font=dict(size=18)))

    if len(envs) == 1:
        annotations.append(dict(xref='paper', yref='paper', x=-0.04, y=0.5,
                                xanchor='center', yanchor='middle',
                                text=envs[0][0], showarrow=False,
                                textangle=-90, font=dict(size=18)))

    fig.update_layout(
        title="",
        legend_title="",
        font=dict(size=13),
        width=1400,
        height=300*len(envs),
        title_x=0.5,
        legend=dict(orientation="h", yanchor="bottom", y=-0.6+len(envs)/5.,xanchor="right",x=0.62),
        annotations=annotations,
        margin=dict(l=70, r=0, b=0, t=50)
    )

    #fig.show()

    fig.write_image(image_name, scale=2)


In [11]:
envs = [
    ("EnergyBoxes", {"average_length": 250, "average_return": 30, "average_eat_count": 45, "average_mix_rate": 1.02, "average_agent_distance": 55}),
    ("EnergyBoxesDelay", {"average_length": 450, "average_return": 150, "average_eat_count": 45, "average_mix_rate": 1.02, "average_agent_distance": 120}),
    ("EnergyBoxesHard", {"average_length": 600, "average_return": 120, "average_eat_count": 120, "average_mix_rate": 1.02, "average_agent_distance": 250})
]

plot_metrics_multiplot(metrics_df, envs, "results-exp2-multiplot-runs.png", plot_runs=True)

In [55]:
envs = [
    ("EnergyBoxes", {"average_length": 250, "average_return": 30, "average_eat_count": 45, "average_mix_rate": 1.02, "average_agent_distance": 55}),
    ("EnergyBoxesDelay", {"average_length": 450, "average_return": 150, "average_eat_count": 45, "average_mix_rate": 1.02, "average_agent_distance": 120}),
    ("EnergyBoxesHard", {"average_length": 600, "average_return": 120, "average_eat_count": 120, "average_mix_rate": 1.02, "average_agent_distance": 250})
]

plot_metrics_multiplot(metrics_df, envs, "results-exp2-multiplot.png")

ValueError: If mode is 'interp', window_length must be less than or equal to the size of x.

In [56]:
envs = [
    ("EnergyBoxesDelay", {"average_length": 450, "average_return": 150, "average_eat_count": 45, "average_mix_rate": 1.02, "average_agent_distance": 120})
]

plot_metrics_multiplot(metrics_df_b, envs, "results-exp2-b-delay.png", plot_runs=True)

In [40]:
timestep_counts = {
    "A": np.load("../minigrid/outputs/timestep_counts-delay-a.npy"),
    "B": np.load("../minigrid/outputs/timestep_counts-delay-b.npy"),
    "Amc": np.load("../minigrid/outputs/timestep_counts-delay-a-mc.npy"),
    "Bmc": np.load("../minigrid/outputs/timestep_counts-delay-b-mc.npy")
}

agent_legend = {"A": "A (goal reward)", "B": "B (time bonus)"}
titles = ["λ=0.95", "λ=1"]
line_colors = {"A": "rgb(128, 128, 128)", "B": "rgb(221, 167, 0)"}

# Create two subplots
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=titles,
    shared_yaxes=False,
)

for agent in ["A", "B"]:

    for mc, col in zip(["", "mc"], [1, 2]):

        fig.add_trace(
            plotly.graph_objects.Scatter(
                x=np.arange(len(timestep_counts[agent+mc])),
                y=timestep_counts[agent+mc],
                line=dict(color=line_colors[agent], width=1),
                name=f"{agent_legend[agent]}",
                showlegend=mc=="",
                mode='lines'  
            ),
            row=1,
            col=col,
        )

        fig.update_yaxes(range=[-10, max(timestep_counts["A"]*1.02)], row=1, col=col)
        fig.update_xaxes(range=[0, len(timestep_counts["A"])], row=1, col=col)

fig.update_layout(
    xaxis_title="Timestep",
    yaxis_title="Eat count",
    width=800,
    height=300,
    margin=dict(l=0, r=0, t=60, b=50)
)

# super title
fig.update_layout(
    title={
        'text': "Eat count per episode timestep",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=18
        )
    }
)

fig.show()
fig.write_image("timestep-count-delay.png", scale=2)


In [18]:
import numpy as np

def plot_runs(df, env_id, metric, image_name="results-exp2-runs-example.png"):
    # Filter the DataFrame by the specific metric and environment ID
    df = df[df['env_id'] == env_id]

    # Create a figure using plotly
    fig = plotly.subplots.make_subplots(rows=1, cols=1)

    # Iterate through the agents and plot the runs, mean, and median
    for agent in ["A", "B"]:
        agent_df = df[df['agent'] == agent]

        for run in sorted(agent_df['run_id'].unique()):
            run_df = agent_df[agent_df['run_id'] == run]
            y_smooth = savgol_filter(run_df[metric], 25, 4)
            fig.add_trace(
                plotly.graph_objects.Scatter(
                    x=run_df['timestep'],
                    y=y_smooth,
                    line=dict(color=line_colors[agent], width=0.8),
                    opacity=0.3,
                    showlegend=False,
                    mode='lines',
                    line_shape='spline',
                )
            )

        # Plot the mean and median
        agent_mean = agent_df.groupby('timestep').mean().reset_index()
        agent_median = agent_df.groupby('timestep').median().reset_index()

        y_mean_smooth = savgol_filter(agent_mean[metric], 25, 4)
        fig.add_trace(
            plotly.graph_objects.Scatter(
                x=agent_mean['timestep'],
                y=y_mean_smooth,
                line=dict(color=line_colors[agent], width=2, dash='dot'),
                name=f"Agent {agent} - Mean",
                mode='lines',
                line_shape='spline',
            )
        )
        y_median_smooth = savgol_filter(agent_median[metric], 25, 4)
        fig.add_trace(
            plotly.graph_objects.Scatter(
                x=agent_median['timestep'],
                y=y_median_smooth,
                line=dict(color=line_colors[agent], width=1),
                name=f"Agent {agent} - Median   ",
                mode='lines',
                line_shape='spline',
            )
        )

    # Update layout and axes
    fig.update_layout(
        title=f"{env_id} - {metrics_legend[metric]}",
        xaxis_title="Timestep",
        yaxis_title="",
        legend_title="",
        font=dict(size=14),
        width=1000,
        height=400,
        title_x=0.5,
        # legend with two columns
        legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="right", x=0.85),
        margin=dict(l=0, r=0, t=50, b=50),
    )
    fig.update_xaxes(range=[0, 20000000])
    fig.update_yaxes(range=[40, 220])

    # Show and save the plot
    fig.show()
    fig.write_image(image_name, scale=2)


In [ ]:
plot_runs(metrics_df, "EnergyBoxes", "average_length")

In [ ]:
def plot_metrics_multiplot_runs(df, envs, image_name, opacity=0.3, width=0.5):

    # use plotly to plot the data
    fig = make_subplots(
        rows=len(envs),
        cols=len(metrics_to_plot),
        subplot_titles=[metrics_legend[metric] for _, _ in envs for metric in metrics_to_plot],
        horizontal_spacing=0.05,
        vertical_spacing=0.1,  # Increase vertical spacing
    )

    for j, (env_id, yaxis_max) in enumerate(envs):
        runs_df = df[df["env_id"]==env_id]

        for i, metric in enumerate(metrics_to_plot):
            for agent in ["A","B"]:
                agent_df = runs_df[runs_df["agent"]==agent]
                showlegend = True
                for run_id in sorted(agent_df['run_id'].unique())[::2]:
                    run_df = agent_df[agent_df['run_id'] == run_id]
                    y_smoothed = savgol_filter(run_df[metric].values, window_length, polyorder)

                    fig.add_trace(
                        plotly.graph_objects.Scatter(
                            x=run_df["timestep"],
                            y=y_smoothed,
                            line=dict(color=line_colors[agent], width=width),
                            name=f"Agent {agent}",
                            mode='lines',
                            line_shape='spline',   
                            showlegend=showlegend,
                            opacity=opacity,
                        ),
                        row=j+1,
                        col=i+1,
                    )

                    if showlegend:
                        showlegend = False
                
            fig.update_yaxes(range=[-0.02*(yaxis_max[metric]), yaxis_max[metric]], row=j+1, col=i+1)
            fig.update_xaxes(range=[0, 20000000], row=j+1, col=i+1, title="Timestep" if j == len(envs) - 1 else None)

    annotations = []
    # Add metric subtitles to the first row
    for i, metric in enumerate(metrics_to_plot):
        annotations.append(dict(xref='paper', yref='paper', x=1.05*(i+0.5)/len(metrics_to_plot)-0.03, y=1.01,
                                xanchor='center', yanchor='bottom',
                                text=metrics_legend[metric], showarrow=False,
                                font=dict(size=18)))
    
    # Add vertical titles
    for j, (env_id, _) in enumerate(envs):
        annotations.append(dict(xref='paper', yref='paper', x=-0.04, y=1.1*((len(envs)-j-0.5)/len(envs))-0.05,
                                xanchor='center', yanchor='middle',
                                text=env_id, showarrow=False,
                                textangle=-90, font=dict(size=18)))

    if len(envs) == 1:
        annotations.append(dict(xref='paper', yref='paper', x=-0.04, y=0.5,
                                xanchor='center', yanchor='middle',
                                text=envs[0][0], showarrow=False,
                                textangle=-90, font=dict(size=18)))

    fig.update_layout(
        title="",
        legend_title="",
        font=dict(size=13),
        width=1400,
        height=300*len(envs),
        title_x=0.5,
        legend=dict(orientation="h", yanchor="bottom", y=-0.6+len(envs)/5.,xanchor="right",x=0.62),
        annotations=annotations,
        margin=dict(l=70, r=0, b=0, t=50)
    )

    fig.show()

    fig.write_image(image_name, scale=2)


In [ ]:
envs = [
    ("EnergyBoxesHard", {"average_length": 600, "average_return": 120, "average_eat_count": 120, "average_mix_rate": 1.02, "average_agent_distance": 250})
]

plot_metrics_multiplot_runs(metrics_df, envs, "results-exp2-hard-runs.png", opacity=0.1)


In [ ]:
envs = [
    ("EnergyBoxes", {"average_length": 200, "average_return": 30, "average_eat_count": 40, "average_mix_rate": 1.02, "average_agent_distance": 50}),
    ("EnergyBoxesDelay", {"average_length": 450, "average_return": 150, "average_eat_count": 45, "average_mix_rate": 1.02, "average_agent_distance": 120})
]

plot_metrics_multiplot_runs(metrics_df, envs, "results-exp2-multiplot-run.png")
